In [2]:
import os
import dill
import numpy as np
import numpyro
from numpyro.infer import Predictive
import arviz as az

import sys
sys.path.append(os.path.pardir)

from gp import prev_model_gp_aggr, args

ImportError: cannot import name 'args' from 'gp' (/hpc/home/idg/workspace/Lo2Hi-IV-Est-AggVAE/notebooks/../gp.py)

In [ ]:
# Load MCMC 
with open("../model_weights/agg_samples1000") as file:
    mcmc = dill.load(file)

In [ ]:
pos_samples = mcmc.get_samples()
mcmc.print_summary(exclude_deterministic = False)

ss = numpyro.diagnostics.summary(mcmc.get_samples(group_by_chain = True))
r = np.mean(ss["gp_aggr"])
print(f"Average ESS for all aggGP effects : {round(r)}")

print(f"kernel_length R-hat : {round(ss['kernel_length']['r_hat'], 2)}")
print(f"kernel_var R-hat : {round(ss['kernel_var']['r_hat'],2)}")


In [1]:
case_samples_az_gp = az.from_numpyro(mcmc)
az.plot_trace(case_samples_az_gp, var_names = ("kernel_length"))
az.plot_trace(case_samples_az_gp, var_names = ("kernel_var"))

NameError: name 'end' is not defined

In [ ]:
# Posterior predictions
case_pos = Predictive(prev_model_gp_aggr, pos_samples)(random.PRNGKey(1), args)

